## Initial data
Initial constants and derived data. Assumes ender pearl is in 1st quadrant

In [6]:
# Allowed coordinate of wind charge (1-axis, including mirrored copies)
windChargeCoord = [
    0.03125,
    0.08375,
    0.09375,
    0.15625,
    0.16625,
    0.20875,
    0.27125,
    0.72875,
    0.79125,
    0.83375,
    0.84375,
    0.90625,
    0.91625,
    0.96875,
]

# Allowed coordinate of pearl (1-axis)
enderPearlCoord = [
    0.0625,
    # 0.24,
    # 0.3025,
]

# TODO - add forbidden pairs of points (i.e. two wall fence pushes - simply can't do them)
#  (use list of tuples, convert into set/dictionary? )

# List of forbidden pairs of coordinates for wind charges (1st quadrant)
forbiddenWindPointList = [
    # Double wall posts are impossible
    (0.91625, 0.91625),
    (0.08375, 0.91625), 
    (0.91625, 0.08375),
    (0.08375, 0.08375),
]


# Max distance from pearl to axis to count them as collinear (units - blocks)
collinearDistThreshold = 0.066
# Max ratio of halves of single axis (longest / shortest, >= 1), to keep pearl centered (units - fraction)
axisCenterRatioError = 1.5
# Max angle error between two axis from 90 deg (units - deg)
axisOrthogErrorDeg = 30
# Max difference between two axis lengths (absolute)
axisLengthError = 0.5
# Minimum safe distance to avoid pearl colliding with charge (along one axis)
axisMinDistThreshold = (0.3125 + 0.25) / 2
# Maximum distance from wind charge to pearl
axisMaxDistThreshold = 0.85

from itertools import product
from shapely import Point

# List of all possible wind charge coords, all 4 quads combined. Filter forbidden points
signedWindCoords = windChargeCoord + [-el for el in windChargeCoord]
windChargePoints = [Point(x, y) for x, y in product(signedWindCoords, repeat=2) if (abs(x),abs(y)) not in forbiddenWindPointList]

# List of all possible ender pearl positions
enderPearlPoints = [Point(x, y) for x, y in product(enderPearlCoord, repeat=2)]

## Axis search
Find all point pairs that cross ender pearl position

In [7]:
from itertools import groupby, product
from shapely import Point, LineString

# Returns True if pearl and charge will collide
def testHitboxCollision(A: Point, B: Point) -> bool:
    dx = abs(A.xy[0][0] - B.xy[0][0])
    dy = abs(A.xy[1][0] - B.xy[1][0])
    return (dx <= axisMinDistThreshold) and (dy <= axisMinDistThreshold)

# Try all combinations of wind charge and ender pearl positions
def getValidAxisList(windPosList: list[Point], pearlPosList: list[Point]) -> list[tuple[LineString, Point]]:
    validLines = []
    for ((windA, windB), pearl) in product(product(windPosList, repeat=2), pearlPosList):
        windLine = LineString([windA, windB])
        pearlDist = windLine.distance(pearl)
        # Test pearl alignment 
        if pearlDist > collinearDistThreshold:
            continue
        distAP = windA.distance(pearl)
        distBP = windB.distance(pearl)
        # Check for collision between wind charge and ender pearl hitboxes
        if testHitboxCollision(windA, pearl) or testHitboxCollision(windB, pearl):
            continue
        # Check for upper distance limit
        if distAP > axisMaxDistThreshold or distBP > axisMaxDistThreshold:
            continue
        # Test ratio of axis halves
        if distBP < 0.0001:
            continue
        distRatio = distAP / distBP
        if distRatio < 1.0:
            if distRatio < 0.0001:
                continue
            distRatio = 1.0 / distRatio
        if distRatio > axisCenterRatioError:
            continue
        # All tests passed, store axis line and associated pearl position
        validLines.append((windLine, pearl))
    return validLines

# Get key string from line and point tuple. Sorts points of line
def getAxisKey(axisEntry: tuple[LineString, Point]):
    linePointText = [Point(ptPair).wkt for ptPair in axisEntry[0].coords]
    linePointText.sort()
    linePointText.append(axisEntry[1].wkt)
    return ";".join(linePointText)

rawAxisList = getValidAxisList(windChargePoints, enderPearlPoints)

# Optimization - sort and remove duplicates (idk if has effect)
rawAxisList.sort(key=getAxisKey)
validAxisList = []
for _,val in groupby(rawAxisList, key=getAxisKey):
    # Only save 1st item in same group
    validAxisList.append(list(val)[0])

## Axis matching
Need to find matching pairs of axis

In [8]:
import numpy as np
from itertools import product
from shapely import Point, LineString

# Get angle between 2 lines. Range - [0; 180] degrees
# (TODO - check)
def angleBetweenLinesDeg(A: LineString, B: LineString) -> float:
    # Convert points to vectors
    u = np.array(A.coords[1]) - np.array(A.coords[0])
    v = np.array(B.coords[1]) - np.array(B.coords[0])
    
    # Compute dot product and magnitudes
    dot_product = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)

    # Compute angle in radians and convert to degrees
    cos_theta = dot_product / (norm_u * norm_v)
    angle_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # Clip for numerical stability
    angle_deg = np.degrees(angle_rad)

    return angle_deg

# Test - count rejections
rejectedByAngle = 0
rejectedByLength = 0

axisCandidates: list[tuple[LineString, LineString, Point]] = []
for ((line1, pearl1), (line2, pearl2)) in product(validAxisList, repeat=2):
    if pearl1 == pearl2:
        pearl = pearl1
        if line1.crosses(line2):
            # See if axis are 90 deg apart
            axisAngleDeg = angleBetweenLinesDeg(line1, line2)
            if abs((abs(axisAngleDeg) - 90)) < axisOrthogErrorDeg:
                # Axis are pretty orthogonal. Compare lengths
                if abs(line1.length - line2.length) < axisLengthError:
                    # Axis are close in length.
                    axisCandidates.append((line1, line2, pearl))
                else:
                    rejectedByLength += 1
            else:
                rejectedByAngle += 1


## Graphical wrapper


In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown
from tabulate import tabulate
from shapely import plotting

def plotResult(index):
    plt.figure(figsize=(6, 6))
    axisPair = axisCandidates[index]
    
    plotting.plot_line(axisPair[0], add_points=True, color="black")
    plotting.plot_line(axisPair[1], add_points=True, color="black")
    plotting.plot_points(axisPair[2], color="red")
        # xVals = (windA[0], windB[0])
        # yVals = (windA[1], windB[1])
        # # Plot solid black line, point marker
        # plotSFig.plot(xVals, yVals, ".-k", linewidth=1)
        # # Plot ender pearl (red plus), every time just in case
        # plotSFig.plot(pearl[0], pearl[1], "+r")
    plt.axhline(0, color="lightgray")
    plt.axvline(0, color="lightgray")
    plt.title(f"Total matches: {len(axisCandidates)}")
    plt.xlabel("X")
    plt.ylabel("Z", rotation=0)
    
    axisTableCombo = ""
    pearlPoint = axisPair[2]
    axisArr = [axisPair[0], axisPair[1]]
    for id, axisLine in enumerate(axisArr):
        axisPoints = list(Point(xyPair) for xyPair in axisLine.coords)
        pointsDist = axisPoints[0].distance(axisPoints[1])
        distPearlToA = axisPoints[0].distance(pearlPoint)
        distPearlToB = axisPoints[1].distance(pearlPoint)
        pearlOffs = pearlPoint.distance(axisLine)
        colLbls = ["Property", "X", "Z"]
        rowNames = ["Point A", "Point B", "Pearl", "Length total", "Length to A", "Length to B", "Pearl offset from line"]
        cellVals = [(axisPoints[0].x, axisPoints[0].y), 
                    (axisPoints[1].x, axisPoints[1].y), 
                    (pearlPoint.x, pearlPoint.y), 
                    (pointsDist, ""), (distPearlToA, ""), (distPearlToB, ""), (pearlOffs, "")]
        combinedRowData = [(descr, *vals) for descr, vals in zip(rowNames, cellVals)]
        axisTable = tabulate(combinedRowData, headers=colLbls, tablefmt="github", floatfmt=".15g")

        axisTableCombo += f"### **Axis {id+1}**" + "\r\n"
        axisTableCombo += axisTable + "\r\n"
    
    plotWidget = widgets.Output(layout={'margin': '20px', 'overflow': 'hidden'})
    with plotWidget:
        plt.show()
    tableWidget = widgets.Output()
    with tableWidget:
        display(Markdown(axisTableCombo))
    dashboard = widgets.HBox([plotWidget, tableWidget])
    display(dashboard)

if len(axisCandidates) == 0:
    print("No matches found!")
else:
    selector = widgets.IntSlider(
        value=0,
        min=0,
        max=len(axisCandidates) - 1,
        step=1,
        description='View Index:',
        continuous_update=False
    )
    axisViewer = widgets.interact(plotResult, index=selector)